#  HASH

---

Визначення:

хеш-функція - приймає обʼєкт що допускає хешування і повертає числове значення.

Операцію отримання для обʼєкта значення хеш-функції інколи називають хешуванням.

Python має вбудовану функцію hash() для хешування обʼєктів.

In [45]:
hash("some string")

-7003137050533436985

In [46]:
hash(False)

0

In [47]:
hash(101)

101

In [48]:
hash(123456789)

123456789

In [49]:
hash(-1)

-2

---

## Деякі твердження про хеш-функції, які можна зустріти в літературі:

### 1. Хеш-функція виконує "незворотнє перетворення" обʼєкта в число.

![irreversible transformation](pictures/hash_obj.png)

тобто: ми можемо отримати значення хеш-функції для обʼєкта, але не можемо відновити обʼєкт з цього значення, навіть знаючи алгоритм хешування.

як Ви бачили з попередніх прикладів, хеш-функція не є "незворотньою" функцією - принаймі для цілих чисел.
Чи означає це що наведене твердження хибне?

Ні. Справа в тому що хеш-функції - це доволі широкий клас алгоритмів, і вони бувають дуже різні. Це твердження справедливо для кріптографічних хеш-функцій. Але бувають і інші.

### 2. Хеш-функція приймає обʼєкт довільного розміру і повертає значення фіксованого розміру.

Доречи - для рядків ми бачимо саме таку поведінку:

In [50]:
hash("a")

8345827218676715222

In [51]:
hash("ababagalamaga")

-6005107276331770660

In [52]:
hash("Python 3.13 is the latest stable release of the Python programming language, with a mix of changes to the language, the implementation and the standard library. The biggest changes include a new interactive interpreter, experimental support for running in a free-threaded mode (PEP 703), and a Just-In-Time compiler (PEP 744).")

-3640777628245663023

але, як Ви бачили з попередніх прикладів, для чисел це не так.

Чому так?

Ви вже знаєте, що кожен обʼєкт в python "несе з собою" всі свої атрибути і можливості. Тобто, наприклад, методи в класі описують поведінку обʼєктів цього класу. І "магічні методи" - описують поведінку обʼєктів при взаємодії з певними операторами або в певних ситуаціях (інколи кажуть - виконуючи певні протоколи).

Існує магічний метод "__hash__" - що описує поведінку обʼєкта при використанні функції hash():

## hash(obj) = obj.\_\_hash\_\_()

Відповідно - для кожного класу обʼєктів може бути власна реалізація методу __hash__ - а значить і свій алгоритм хешування.

А ці алгоритми можуть бути дуже різними - як ми з вами вже знаємо.

### 3. "Значення хеш-функції для однакових обʼєктів завжди однакове. Завдяки "незворотності перетворення" хешування паролів користувачів дає можливість зберегти в БД не паролі, а їх хеш-значення. Якщо зловмисник отримає доступ до вашої БД - він не зможе отримати паролі, а лише їх хеш-значення - які вже не можна відновити в паролі."

In [53]:
hash("пароль що я використовую часто")

-9146907316442277749

![auth form](pictures/auth_form_hash.png)

показати приклади з використанням хеш-функцій в різних процесах python

[документація про hash](https://docs.python.org/3/glossary.html#term-hashable)

Python гарантує однакове значення hash-функції для одного обʼєкту лише в період існування цього обʼєкту в межах одного процесу python. При перезапуску процесу - значення hash може бути іншим.

А як же тоді історія з паролями? Хіба це не правда?

Ні, це абсолютна правда. Саме так і працюють з паролями. Але ми вже побачили що алгоритми для створення hash-функцій бувають різні. І для обробки паролів використовують інші криптографічні hash-функції. Наприклад - з бібліотеки [hashlib](https://docs.python.org/3/library/hashlib.html).

###  Що таке "хешуваний обʼєкт" і для яких обʼєктів може бути визначена hash-функція?

---

- більшість незмінних вбудованих об’єктів Python можна хешувати;
- змінні контейнери (такі як списки або словники, множини) ні;
- незмінні контейнери (такі як кортежі та заморожені множини) можна хешувати, лише якщо їх елементи хешуються.

In [54]:
hash([1, 2, 3])

TypeError: unhashable type: 'list'

In [55]:
hash((1, 2, 3))

529344067295497451

In [56]:
hash((1, 2, [1, 2]))

TypeError: unhashable type: 'list'

---

## якщо обʼєкт можна хешувати - він може бути ключем словника або елементом множини.

#### P.S. недарма словники називаються "хеш-таблицями" - вони використовують хеш-значення ключів для швидкого доступу до значень. Але це тема для іншого відеоролика. Буде і такий - підписуйтесь.

---

## Хеш-функції для користувацьких класів

###  Поведінка за замовченням.

In [57]:
class User:
    def __init__(self, username: str, email: str) -> None:
        self.username = username
        self.email = email

    def __repr__(self):
        return f"User({self.username}, {self.email})"

In [58]:
user_1 = User("test_1", "test_1@test.test")
user_2 = User("test_2", "test_2@test.test")

In [59]:
hash(user_1), hash(user_2)

(273402617, 274273585)

In [60]:
user_1.__hash__()

273402617

За замовченням кожен користувацький клас має реалізацію методу __hash__ яка використовується для хешування обʼєктів класу. Ця реалізація використовує id() обʼєкта - тобто для кожного обʼєкта класу буде використано його унікальний ідентифікатор.

Якщо така поведінка не підходить - можна перевизначити метод __hash__ для класу.

Наприклад: ми пишемо сайт і у нас користувачі розрізняються по їх email. Це є унікальний ідентифікатор користувача і двох однакових бути не може. Тобто - ми вважаємо що користувачі різні, якщо вони мають різні email. Тоді можна перевизначити метод __hash__ для класу User:

In [61]:
user_1_copy = User("test_1", "test_1@test.test")
hash(user_1_copy), hash(user_1)

(273283397, 273402617)

In [62]:
class User:
    def __init__(self, username: str, email: str) -> None:
        self.username = username
        self.email = email

    def __repr__(self):
        return f"User({self.username}, {self.email})"

    def __hash__(self):
        return hash(self.email)

тепер ми можемо екземпляри класу User використовувати як ключі в словниках або елементи множини:

In [63]:
user1 = User("USER1", "user1@user.user")
user2 = User("user1", "user1@user.user")
user1 is user2

False

In [64]:
hash(user1), hash(user2)

(4509677507545669266, 4509677507545669266)

In [65]:
users_set = set()
users_set.add(user1)
users_set.add(user2)
users_set

{User(USER1, user1@user.user), User(user1, user1@user.user)}

Справа в тому, що визначення унікальності в Python має більш складну природу. Незважаючи на те, що ці обʼєкти мають однаковий хеш - вони для нас не є рівними:

In [66]:
user1 == user2

False

Тому що ми не перевизначали метод __eq__ для класу User. І за замовченням він використовує порівняння по id() обʼєкта. І для двох обʼєктів класу User - це завжди будуть різні обʼєкти, а значить вони не рівні. Є ще одно правило якеого треба притримуватись якщо Ви бажаєте очікуваної поведінки від своїх обʼєктів:

## Якщо a == b, то hash(a) == hash(b)

недостатньо однакових значень хешу - обʼєкти повинні дорівнювати один одному:

In [67]:
class User:
    def __init__(self, username: str, email: str) -> None:
        self.username = username
        self.email = email

    def __repr__(self):
        return f"User({self.username}, {self.email})"

    def __hash__(self):
        return hash(self.email)

    def __eq__(self, other):
        return isinstance(other, type(self)) and self.email == other.email

In [68]:
user1 = User("USER1", "user1@user.user")
user2 = User("user1", "user1@user.user")
user1 is user2

False

In [69]:
hash(user1), hash(user2)

(4509677507545669266, 4509677507545669266)

In [70]:
user1 == user2

True

In [71]:
users_set = set()
users_set.add(user1)
users_set.add(user2)
users_set

{User(USER1, user1@user.user)}

и тепер ми можем використовувати обʼєкти класу User як ключі в словниках або елементи множини - і вони не будуть дублюватись

In [72]:
users_dict = {user1: "user1", user2: "user2"}

In [73]:
users_dict

{User(USER1, user1@user.user): 'user2'}

Тобто: правильною поведінкою буде визначати метод __eq__ для класу, якщо Ви перевизначаєте метод __hash__:

## Якщо a == b, то hash(a) == hash(b)

але це не все)

In [74]:
user1 in users_set, user1 in users_dict

(True, True)

In [75]:
user1.email = 'new_email@test.test'

In [76]:
user1 in users_set, user1 in users_dict

(False, False)

In [77]:
users_dict

KeyError: User(USER1, new_email@test.test)

In [78]:
users_set

{User(USER1, new_email@test.test)}

ще одна важлива умова:

##  Хеш обʼєкта ніколи не повинен змінюватись протягом життєвого циклу цього обʼєкту

* ми говоримо саме про той хеш, що використовується в хеш-таблицях (словниках, множинах)

In [79]:
class User:
    def __init__(self, username: str, email: str) -> None:
        self.username = username
        self._email = email

    @property  #  це декоратор для створення властивостей класу. Поговоримо детально про це пізніше. Підписуйтесь)
    def email(self):
        return self._email

    @email.setter
    def email(self, value):
        raise TypeError(f"{self.__class__.__name__} object is immutable. Can't change email.")

    def __repr__(self):
        return f"User({self.username}, {self.email})"

    def __hash__(self):
        return hash(self.email)

    def __eq__(self, other):
        if not isinstance(other, type(self)):
            return NotImplemented
        return self.email == other.email

In [80]:
user1 = User("USER1", "user1@user.user")

In [81]:
user1.email = "new_email@test.test"

TypeError: User object is immutable. Can't change email.

### Важливо пам'ятати: всі ці складні конструкції треба використовувати лише тоді коли Ви точно розумієте що це потрібно: ви дійсно плануєте використовувати обʼєкти класу як ключі в словниках або елементи множини. Інакше - це може призвести до непотрібної складності коду і зниження продуктивності. Ніколи не пишіть код "на виріст".